In [1]:
import pandas as pd
import numpy as np 

In [4]:
import os
for dirname, _, filenames in os.walk(r"C:\python\Recommender-system\data"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\python\Recommender-system\data\credits.csv
C:\python\Recommender-system\data\keywords.csv
C:\python\Recommender-system\data\movies_metadata.csv
C:\python\Recommender-system\data\ratings.csv


In [5]:
item = pd.read_csv('C:\python\Recommender-system\data\movies_metadata.csv')

C:\Users\ACER\AppData\Local\Temp\ipykernel_594612\143846674.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  item = pd.read_csv('C:\python\Recommender-system\data\movies_metadata.csv')


In [6]:
item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [7]:
item = item[['id']]
item

,id
0,862
1,8844
2,15602
3,31357
4,11862
...,...
45461,439050
45462,111109
45463,67758
45464,227506


In [13]:
id_to_index = {original_id: idx + 1 for idx, original_id in enumerate(item['id'])}

item['new_id'] = item['id'].map(id_to_index)

C:\Users\ACER\AppData\Local\Temp\ipykernel_594612\563534749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item['new_id'] = item['id'].map(id_to_index)


In [14]:
item

,id,new_id
0,862,1
1,8844,2
2,15602,3
3,31357,4
4,11862,5
...,...,...
45461,439050,45459
45462,111109,45460
45463,67758,45461
45464,227506,45462


In [15]:
from datetime import datetime

ratings_df = pd.read_csv(r"C:\python\Recommender-system\data\ratings.csv")

In [16]:
ratings_df['rating'] = ratings_df['rating'].astype(int)
ratings_df = ratings_df[['userId', 'movieId', 'rating']]
ratings_df

,userId,movieId,rating
0,1,110,1
1,1,147,4
2,1,858,5
3,1,1221,5
4,1,1246,5
...,...,...,...
26024284,270896,58559,5
26024285,270896,60069,5
26024286,270896,63082,4
26024287,270896,64957,4


In [17]:
item['id'] = pd.to_numeric(item['id'], errors='coerce')

item = item.dropna(subset=['id'])

item['id'] = item['id'].astype(int)

ratings_df = ratings_df.merge(item[['id', 'new_id']], how='left', left_on='movieId', right_on='id')

ratings_df = ratings_df.drop(columns=['id'])

ratings_df = ratings_df.dropna(subset=['new_id'])

ratings_df['new_id'] = ratings_df['new_id'].astype(int)

ratings_df

C:\Users\ACER\AppData\Local\Temp\ipykernel_594612\1382422914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item['id'] = pd.to_numeric(item['id'], errors='coerce')


,userId,movieId,rating,new_id
0,1,110,1,303
1,1,147,4,2617
2,1,858,5,535
4,1,1246,5,11447
5,1,1968,4,1401
...,...,...,...,...
26025338,270896,48780,5,43366
26025340,270896,49530,4,17981
26025347,270896,54001,4,16935
26025349,270896,54503,4,30673


In [18]:
ratings = ratings_df[['userId', 'new_id', 'rating']]
ratings = ratings.iloc[:500000, :]
ratings

,userId,new_id,rating
0,1,303,1
1,1,2617,4
2,1,535,5
4,1,11447,5
5,1,1401,4
...,...,...,...
1141682,11743,1296,2
1141684,11743,41691,3
1141687,11743,2446,4
1141688,11743,785,2


In [19]:
n_users = ratings['userId'].nunique()
print(n_users)
n_items = ratings['new_id'].nunique()
print(n_items)

11528
5147


In [20]:
train_list = []
test_list = []

for user_id, group in ratings.groupby('userId'):

    test_row = group.sample(n=1, random_state=42)
    test_list.append(test_row)

    train_rows = group.drop(test_row.index)
    train_list.append(train_rows)

train_df = pd.concat(train_list)
test_df = pd.concat(test_list)

print(f'Số lượng hàng trong tập train: {len(train_df)}')
print(f'Số lượng hàng trong tập test: {len(test_df)}')

Số lượng hàng trong tập train: 488472
Số lượng hàng trong tập test: 11528


In [21]:
rate_train = train_df.values
rate_test = test_df.values

In [22]:
print(rate_test)

[[    1  2101     4]
 [    2    18     3]
 [    3  1058     2]
 ...
 [11741  5005     4]
 [11742  8829     5]
 [11743  3824     2]]


In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

class CF(object):
    def __init__(self, Y_data, k, dist_func=cosine_similarity, uuCF=1):
        self.uuCF = uuCF  
        self.Y_data = Y_data if uuCF else Y_data[:, [1, 0, 2]]
        self.k = k  
        self.dist_func = dist_func
        self.Ybar_data = None
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1

    def add(self, new_data):
        self.Y_data = np.concatenate((self.Y_data, new_data), axis=0)

    def normalize_Y(self):
        users = self.Y_data[:, 0]  
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            ids = np.where(users == n)[0].astype(np.int32)
            item_ids = self.Y_data[ids, 1]
            ratings = self.Y_data[ids, 2]
            m = np.mean(ratings)
            if np.isnan(m):
                m = 0  
            self.Ybar_data[ids, 2] = ratings - self.mu[n]
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
                                       (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
        self.Ybar = self.Ybar.tocsr()

    def similarity(self):
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)

    def refresh(self):
        self.normalize_Y()
        self.similarity()

    def fit(self):
        self.refresh()

    def __pred(self, u, i, normalized=1):
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        sim = self.S[u, users_rated_i]
        a = np.argsort(sim)[-self.k:]
        nearest_s = sim[a]
        r = self.Ybar[i, users_rated_i[a]]
        if normalized:
            return (r * nearest_s)[0] / (np.abs(nearest_s).sum() + 1e-8)

        return (r * nearest_s)[0] / (np.abs(nearest_s).sum() + 1e-8) + self.mu[u]

    def pred(self, u, i, normalized=1):
        if self.uuCF:
            return self.__pred(u, i, normalized)
        return self.__pred(i, u, normalized)


In [24]:
import numpy as np

def recommend_top_movies(data, train_items, user_id, rs, top_n=5):
    items_rated_by_user = set(data[data[:, 0] == user_id, 1])

    predicted_ratings = []
    for item_id in train_items:
        if item_id not in items_rated_by_user:
            pred = rs.pred(user_id, item_id, normalized=0) 
            pred = np.clip(pred, 0, 5)  
            predicted_ratings.append((item_id, pred))

    predicted_ratings = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)

    return predicted_ratings[:top_n]

In [25]:
rs = CF(rate_train, k=30, uuCF=1)
rs.fit()
train_items = set(rate_train[:, 1])

def calculate_metrics(data, train_items):
    n_samples = data.shape[0]
    SE = 0 
    AE = 0 
    valid_samples = 0  

    for n in range(n_samples):
        user_id, item_id, rating = data[n]
        if item_id not in train_items:
            continue

        pred = rs.pred(user_id, item_id, normalized=0)
        SE += (pred - rating) ** 2
        AE += abs(pred - rating)
        valid_samples += 1

    if valid_samples > 0:
        MSE = SE / valid_samples 
        RMSE = np.sqrt(MSE)
        MAE = AE / valid_samples 
        return RMSE, MSE, MAE
    else:
        return None, None, None
    
rmse_train, mse_train, mae_train = calculate_metrics(rate_train, train_items)
if rmse_train is not None:
    print(f"RMSE trên tập huấn luyện: {rmse_train:.4f}")
    print(f"MSE  trên tập huấn luyện: {mse_train:.4f}")
    print(f"MAE  trên tập huấn luyện: {mae_train:.4f}")

rmse_test, mse_test, mae_test = calculate_metrics(rate_test, train_items)
if rmse_test is not None:
    print(f"RMSE trên tập kiểm tra  : {rmse_test:.4f}")
    print(f"MSE  trên tập kiểm tra  : {mse_test:.4f}")
    print(f"MAE  trên tập kiểm tra  : {mae_test:.4f}")


c:\Users\ACER\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\ACER\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


RMSE trên tập huấn luyện: 0.8594
MSE  trên tập huấn luyện: 0.7386
MAE  trên tập huấn luyện: 0.6656
RMSE trên tập kiểm tra  : 1.2608
MSE  trên tập kiểm tra  : 1.5896
MAE  trên tập kiểm tra  : 0.9187


In [26]:
user_id = 2  
top_n = 5    

recommended_movies = recommend_top_movies(rate_test, train_items, user_id, rs, top_n)

print(f"Gợi ý {top_n} phim hàng đầu cho người dùng {user_id}:")
for movie_id, rating in recommended_movies:
    print(f"- Movie ID: {movie_id}, Rating dự đoán: {rating:.2f}")

Gợi ý 5 phim hàng đầu cho người dùng 2:
- Movie ID: 33825, Rating dự đoán: 5.00
- Movie ID: 35796, Rating dự đoán: 5.00
- Movie ID: 19049, Rating dự đoán: 5.00
- Movie ID: 1424, Rating dự đoán: 5.00
- Movie ID: 44454, Rating dự đoán: 5.00


In [27]:
rs = CF(rate_train, k=30, uuCF=0)
rs.fit()
train_items = set(rate_train[:, 1])

def calculate_metrics(data, train_items):
    n_samples = data.shape[0]
    SE = 0  
    AE = 0 
    valid_samples = 0 

    for n in range(n_samples):
        user_id, item_id, rating = data[n]
        if item_id not in train_items:
            continue

        pred = rs.pred(user_id, item_id, normalized=0)
        SE += (pred - rating) ** 2
        AE += abs(pred - rating)
        valid_samples += 1

    if valid_samples > 0:
        MSE = SE / valid_samples 
        RMSE = np.sqrt(MSE)  
        MAE = AE / valid_samples 
        return RMSE, MSE, MAE
    else:
        return None, None, None

rmse_train, mse_train, mae_train = calculate_metrics(rate_train, train_items)
if rmse_train is not None:
    print(f"RMSE trên tập huấn luyện: {rmse_train:.4f}")
    print(f"MSE  trên tập huấn luyện: {mse_train:.4f}")
    print(f"MAE  trên tập huấn luyện: {mae_train:.4f}")

rmse_test, mse_test, mae_test = calculate_metrics(rate_test, train_items)
if rmse_test is not None:
    print(f"RMSE trên tập kiểm tra  : {rmse_test:.4f}")
    print(f"MSE  trên tập kiểm tra  : {mse_test:.4f}")
    print(f"MAE  trên tập kiểm tra  : {mae_test:.4f}")


RMSE trên tập huấn luyện: 0.7857
MSE  trên tập huấn luyện: 0.6173
MAE  trên tập huấn luyện: 0.6016
RMSE trên tập kiểm tra  : 1.2723
MSE  trên tập kiểm tra  : 1.6187
MAE  trên tập kiểm tra  : 0.9141


In [28]:
user_id = 2  
top_n = 5    

recommended_movies = recommend_top_movies(rate_test, train_items, user_id, rs, top_n)

print(f"Gợi ý {top_n} phim hàng đầu cho người dùng {user_id}:")
for movie_id, rating in recommended_movies:
    print(f"- Movie ID: {movie_id}, Rating dự đoán: {rating:.2f}")


Gợi ý 5 phim hàng đầu cho người dùng 2:
- Movie ID: 582, Rating dự đoán: 5.00
- Movie ID: 1206, Rating dự đoán: 5.00
- Movie ID: 17348, Rating dự đoán: 5.00
- Movie ID: 33512, Rating dự đoán: 4.27
- Movie ID: 8624, Rating dự đoán: 4.27
